<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/50/Logo_fh_hof.svg/2000px-Logo_fh_hof.svg.png" width="250" style="background-color:#FFF">

---

<div style="text-align:center;">
    <font size="+4"><i><u>Q-Learning mit 'Super Mario Bros'</u></i></font><br><br>
    <font>Seminararbeit der Vorlesung <b>Angewandtes Maschinelles Lernen</b> an der <b>Hochschule für angewande Wissenschaften Hof</b> des <b>Sommersemesters 2020</b>.</font>
</div>

---

<h1>Inhaltsverzeichnis<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Parameter-&amp;-Hilfsunktionen" data-toc-modified-id="Parameter-&amp;-Hilfsunktionen-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Parameter &amp; Hilfsunktionen</a></span></li><li><span><a href="#Memory" data-toc-modified-id="Memory-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Memory</a></span></li></ul></div>

---

## Imports

In [ ]:
import torch
import numpy as np

---

## Parameter & Hilfsunktionen

In [ ]:
_memory_device = get_torch_device()

print("Memory-Device-Parameter initialisiert ...")

---

## Memory

In [ ]:
class Memory:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, state_size, action_size, buffer_size, batch_size, priority = False):
        """Initialize a ReplayBuffer object.
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer (chosen as multiple of num agents)
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.states = torch.zeros((buffer_size,) + state_size).to(_memory_device)
        self.next_states = torch.zeros((buffer_size,) + state_size).to(_memory_device)
        self.actions = torch.zeros(buffer_size,1, dtype = torch.long).to(_memory_device)
        self.rewards = torch.zeros(buffer_size, 1, dtype = torch.float).to(_memory_device)
        self.dones = torch.zeros(buffer_size, 1, dtype = torch.float).to(_memory_device)
        self.e = np.zeros((buffer_size, 1), dtyp e= np.float)
        
        self.priority = priority

        self.ptr = 0
        self.n = 0
        self.buffer_size = buffer_size
        self.batch_size = batch_size
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        self.states[self.ptr] = torch.from_numpy(state).to(_memory_device)
        self.next_states[self.ptr] = torch.from_numpy(next_state).to(_memory_device)
        self.actions[self.ptr] = action
        self.rewards[self.ptr] = reward
        self.dones[self.ptr] = done
        
        self.ptr += 1
        if self.ptr >= self.buffer_size:
            self.ptr = 0
            self.n = self.buffer_size

    def sample(self, get_all=False):
        """Randomly sample a batch of experiences from memory."""
        n = len(self)
        if get_all:
            return self.states[:n], self.actions[:n], self.rewards[:n], self.next_states[:n], self.dones[:n]
        # else:
        if self.priority:
            idx = np.random.choice(n, self.batch_size, replace=False, p=self.e)
        else:
            idx = np.random.choice(n, self.batch_size, replace=False)
        
        states = self.states[idx]
        next_states = self.next_states[idx]
        actions = self.actions[idx]
        rewards = self.rewards[idx]
        dones = self.dones[idx]
        
        return (states, actions, rewards, next_states, dones), idx
      
    def update_error(self, e, idx=None):
        e = torch.abs(e.detach())
        e = e / e.sum()
        if idx is not None:
            self.e[idx] = e.cpu().numpy()
        else:
            self.e[:len(self)] = e.cpu().numpy()
        
    def __len__(self):
        if self.n == 0:
            return self.ptr
        else:
            return self.n